In [ ]:
# Web Scraper - Capstone Project

In [ ]:
# Run to install modules
! pip install fake_useragent
! pip install bs4
! pip install seaborn

In [ ]:
# Import the modules in PIP8 Style
import requests
import pandas as pd
import bs4
import numpy as np
import seaborn as sns

from fake_useragent import UserAgent

In [ ]:
products = [] # List to store name of the product
prices = [] # List to store price of the product
ratings = [] # List to store rating of the product
specifications = [] # List to store specifications of the product
sim_types = []
colors = []
storage_sizes = []
ram_sizes = []
battery_capacities = []
operating_systems = []
df = pd.DataFrame() # Products to be stored in a Pandas dataframe

index=0

In [ ]:
# Creating a fake user agent
user_agent = UserAgent()

In [ ]:
# Get a product name from User
product_name = "mobile phone"

In [ ]:
for i in range(1,2):
    url = "https://www.flipkart.com/search?q={0}&page={1}"
    url = url.format(product_name, i)
    # getting the reponse from the page using get method of requests module
    page = requests.get(url, headers = {"user-agent": user_agent.firefox})
    # storing the content of the page in a variable
    html = page.content
    page.close()
    # creating BeautifulSoup object
    strainer = bs4.SoupStrainer("div", attrs={"class": "_1YokD2 _3Mn1Gg", "style":"flex-grow:1;overflow:auto"})
    soup = bs4.BeautifulSoup(html, "html.parser", parse_only = strainer)
    # Using list of class names as Flipkart uses two different card components for presenting products.
    # "class" : [List View Card, Grid View Card]
    for containers in soup.find_all("div", attrs = {"class" : "_2kHMtA"}):
        index+=1
        product_url = containers.find('a', attrs = {'class':'_1fQZEK'}).get('href')
        product_url = "https://flipkart.com" + product_url
        product_page = requests.get(product_url, headers = {"user-agent": user_agent.firefox})
        product_html = product_page.content
        product_page.close()
        product_strainer = bs4.SoupStrainer("div", attrs={"class": "_1YokD2 _3Mn1Gg col-8-12"})
        product_soup = bs4.BeautifulSoup(product_html, "html.parser", parse_only=product_strainer)
        for specs in product_soup.find_all("div", attrs = {"class": "_1UhVsV"}):
            # Get SIM Type
            product_sim_type = specs.find(attrs = {"class": "_1hKmbr"}, string = "SIM Type")
            if (type(product_sim_type) == bs4.element.Tag):
                sim_types.append(product_sim_type.find_next_sibling().li.text)
                #print(product_sim_type.text)
            else:
                sim_types.append('NaN')

                
            # Get Color
            color = specs.find(attrs = {"class": "_1hKmbr"}, string = "Color")
            if (type(color) == bs4.element.Tag):
                colors.append(color.find_next_sibling().li.text.split(" ")[1])
                #print(color.text)
            else:
                colors.append('NaN')
                
            # Get RAM Size
            ram = specs.find(attrs = {"class": "_1hKmbr"}, string = "RAM")
            if (type(ram) == bs4.element.Tag):
                ram_sizes.append(ram.find_next_sibling().li.text.split(" ")[0])
                #print(ram.text)
            else:
                ram_sizes.append('NaN')
                
            # Get Storage Size
            storage = specs.find(attrs = {"class": "_1hKmbr"}, string = "Internal Storage")
            if (type(storage) == bs4.element.Tag):
                storage_sizes.append(storage.find_next_sibling().li.text.split(" ")[0])
                #print(storage.text)
            else:
                storage_sizes.append('NaN')
            
            # Get Battery Capacity
            battery = specs.find(attrs = {"class": "_1hKmbr"}, string = "Battery Capacity")
            if (type(battery) == bs4.element.Tag):
                battery_capacities.append(battery.find_next_sibling().li.text.split(" ")[0])
                #print(battery.text)
            else:
                battery_capacities.append('NaN')
            
            # Get Operating System
            operating_system = specs.find(attrs = {"class": "_1hKmbr"}, string = "Operating System")
            if (type(operating_system) == bs4.element.Tag):
                s = operating_system.find_next_sibling().li.text
                s1 = (' '.join(dict.fromkeys(s.split())))
                operating_systems.append(s1)
            else:
                operating_systems.append('NaN')

        # Saving name of the product in a variable using the find method
        name = containers.find(attrs = {"class" : "_4rR01T"})
        products.append(name.text.split('(')[0])
        price = containers.find("div", attrs = {"class" : "_1_WHN1"})
        prices.append(price.text.split("₹")[1].replace(',', ''))
        rating = containers.find('div', attrs = {'class':'_3LWZlK'})
        if (type(rating) == bs4.element.Tag):
            ratings.append(rating.text)
        else:
            ratings.append('NaN')
        dtypes = {"Product Name": "object", "Price": np.int32, "SIM": "category",
                  "RAM": np.int32, "OS": "category", "Internal Storage": np.int32, "Colors": "category", "Battery Capacity": np.int32, "Rating": np.float32}
        df = pd.DataFrame({'Product Name':products, 'Price':prices, 'SIM':sim_types, 'RAM':ram_sizes, 'OS':operating_systems, 'Internal Storage':storage_sizes, 'Colors':colors, 'Battery Capacity':battery_capacities, 'Rating':ratings}) 
        df = df.astype(dtypes)
        print(index)
df.info()
df

In [ ]:
sorted_df = df.sort_values(by=['Price'], ascending=True)

sorted_df
sns.barplot(x=sorted_df["RAM"], y=sorted_df["Price"])

In [ ]:
sns.lineplot(df['RAM'], df['Price'], x= 'RAM', y = 'Price')

In [ ]:
sorted_df.info()